<a href="https://colab.research.google.com/github/sweetPototo/Data-Analysis-with-Open-Source/blob/main/energyConsumptionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 : 서울시 에코마일리지 에너지사용량 통계정보

1-2. 오픈소스 연결 및 실행결과

In [ ]:
import requests
import time

api_key = "6d4b686f767368703131395a4e4f644d"
data = []

for year in range(2015, 2025):
    for mon in range(1,13):
        mon = f"{mon:02}"
        url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataSummaryInfo/1/5/{year}/{mon}'
        response = requests.get(url)

        if response.status_code == 200:
            res_json = response.json()
            rows = res_json.get("energyUseDataSummaryInfo",{}).get("row",{})
            for row in rows:
                if row.get("MM_TYPE") == "개인":
                    data.append({
                        "YEAR":row.get("YEAR"),
                        "MON":row.get("MON"),
                        "MM_TYPE":row.get("MM_TYPE"), #회원타입
                        "EUS":row.get("EUS"),         #현년 전기사용량
                        "GUS":row.get("GUS"),         #현년 가스사용량
                        "WUS":row.get("WUS"),         #현년 수도사용량
                        "HUS":row.get("HUS")          #현년 지역난방 사용량
                    })
        else:
            print(f"API 호출 실패: {response.status_code}")
        time.sleep(0.3)

    print(f"{year}년 데이터 적재 완료")
print("수집 결과")
print(data)


2015 데이터 적재 완료
2016 데이터 적재 완료
2017 데이터 적재 완료
2018 데이터 적재 완료
2019 데이터 적재 완료
2020 데이터 적재 완료
2021 데이터 적재 완료
2022 데이터 적재 완료
2023 데이터 적재 완료
2024 데이터 적재 완료
수집 결과
[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'EUS': '193784708', 'GUS': '59133720', 'WUS': '12819757.886', 'HUS': '22740838.937'}, {'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'EUS': '189974230', 'GUS': '56487358', 'WUS': '12656888.218', 'HUS': '18793320.213'}, {'YEAR': '2015', 'MON': '05', 'MM_TYPE': '개인', 'EUS': '177289769', 'GUS': '22060892', 'WUS': '13086734.214', 'HUS': '1320895.801'}, {'YEAR': '2015', 'MON': '06', 'MM_TYPE': '개인', 'EUS': '186073153', 'GUS': '12677645', 'WUS': '14241651.402', 'HUS': '294312.634'}, {'YEAR': '2015', 'MON': '07', 'MM_TYPE': '개인', 'EUS': '165114568', 'GUS': '8893640', 'WUS': '12083879.038', 'HUS': '702863.645'}, {'YEAR': '2015', 'MON': '08', 'MM_TYPE': '개인', 'EUS': '242000479', 'GUS': '7619018', 'WUS': '15164553.742', 'HUS': '460288.965'}, {'YEAR': '2015', 'MON': '10', 'MM_TYPE': '개인', 'EUS': 

2-1. DataFrame으로 변경 및 정보 출력

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   YEAR     83 non-null     object
 1   MON      83 non-null     object
 2   MM_TYPE  83 non-null     object
 3   EUS      83 non-null     object
 4   GUS      83 non-null     object
 5   WUS      83 non-null     object
 6   HUS      83 non-null     object
dtypes: object(7)
memory usage: 4.7+ KB


2-2. 연도 및 계절 컬럼 추가

In [ ]:
energy_use = df.copy()
energy_use['year'] = df['YEAR']
energy_use['season'] = energy_use['MON'].map({'01':'겨울', '02':'겨울', '03':'봄',
                        '04':'봄', '05':'봄', '06':'여름', '07':'여름', '08':'여름',
                                '09':'가을', '10':'가을', '11':'가을', '12':'겨울'})
print(energy_use)

    YEAR MON MM_TYPE        EUS       GUS           WUS           HUS  year  \
0   2015  01      개인  193784708  59133720  12819757.886  22740838.937  2015   
1   2015  02      개인  189974230  56487358  12656888.218  18793320.213  2015   
2   2015  05      개인  177289769  22060892  13086734.214   1320895.801  2015   
3   2015  06      개인  186073153  12677645  14241651.402    294312.634  2015   
4   2015  07      개인  165114568   8893640  12083879.038    702863.645  2015   
..   ...  ..     ...        ...       ...           ...           ...   ...   
78  2024  08      개인  463971328   7072251    15873544.6    130082.263  2024   
79  2024  09      개인  409872908   5729131    15143321.6   3420507.136  2024   
80  2024  10      개인  282268119   8233421    15297669.9   2695491.396  2024   
81  2024  11      개인  257927956  17418221    14678672.5   8836296.454  2024   
82  2024  12      개인  134828126  42103424     7326853.6   1513199.277  2024   

   season  
0      겨울  
1      겨울  
2       봄  
3  

3-1. 데이터 시각화